# 光学シミュレーション

## 参考文献
- [光学機能性フィルムのシミュレーションによる設計と実験的検証](https://www.kobelco.co.jp/technology-review/pdf/65_2/050-053.pdf)
- [光学薄膜の分光特性シミュレーション](https://www.jstage.jst.go.jp/article/sfj1989/48/9/48_9_890/_pdf)
- [基礎物理定数|ウシオ電機](https://www.ushio.co.jp/jp/technology/glossary/material/attached_material_03.html)

## スネルの法則
$$
n_{0}\sin\theta_{0}=n_{1}\sin\theta_{1} \\
\theta_{1} = \sin^{-1}\left(\frac{n_{0}}{n_{1}}\sin\theta_{0}\right)
$$ 

In [1]:
# import
import numpy as np

In [2]:
# code
def snell(n1,n0,theta):
    return np.arcsin(n0.real/n1.real*np.sin(theta))


In [3]:
# test
snell(1+1j,1,0)

0.0

## 光学アドミタンス
$$
\eta =
\left\{
    \begin{array}{ll}
        Y_{0}n\cos\phi & \text{s偏光} \\
        \displaystyle\frac{Y_{0}n}{\cos\phi} & \text{p偏光}
    \end{array}
\right.
$$
ここで$Y_{0}$は真空の光学アドミタンスであり
$$
Y_{0} =\sqrt{\frac{\epsilon_{0}}{\mu_{0}}}
$$
$\epsilon_{0}$:真空の誘電率、$\mu_{0}$:真空の透磁率である。



In [4]:
# Constants
EPS = 8.8541878128e-12
MU = 1.25663706212e-6
Y0 = np.sqrt(EPS/MU)

In [5]:
# code
def admittance(n1,n0,theta):
    phi = snell(n1,n0,theta)
    return (Y0*n1*np.cos(phi), Y0*n1/np.cos(phi))

In [6]:
# test
admittance(1.0,1.0,0.0*np.pi/180)==(Y0,Y0)

True

## 位相差
$$
2\delta = \frac{4\pi}{\lambda}nd\cos\phi
$$

In [7]:
# code
def phasedifference(n,d,phi,lm):
    return 2*np.pi*n*d*np.cos(phi)/lm

In [8]:
# test
phasedifference(2.0,125,0,500)

3.141592653589793

## 特性マトリクス
$$
M_{i} = \left(
\begin{array}{cc}
\cos\delta_{i} & i\eta_{i}^{-1}\sin\delta_{i} \\
i\eta_{i}\sin\delta_{i} & \cos\delta_{i}
\end{array}
\right)
$$


In [9]:
# code
def characteristic_matrix(n,d,n0,angle,lam):
    eta = admittance(n,n0,angle)
    phi = snell(n,n0,angle)
    delta = phasedifference(n,d,phi,lam)
    c = np.cos(delta)
    s = np.sin(delta)
    def _matrix(et):
        return (np.array([[c,1j*s/et],[1j*s*et,c]]))
    return tuple(map(_matrix, eta))

In [10]:
# test
characteristic_matrix(1.0,50,1.0,0,550)

(array([[0.84125353+0.00000000e+00j, 0.        +2.03675785e+02j],
        [0.        +1.43508711e-03j, 0.84125353+0.00000000e+00j]]),
 array([[0.84125353+0.00000000e+00j, 0.        +2.03675785e+02j],
        [0.        +1.43508711e-03j, 0.84125353+0.00000000e+00j]]))

## 反射
$$
R = \left|\frac{\eta_{0}B-C}{\eta_{0}B+C}\right|^{2}
$$

In [11]:
# code
def reflectance(param, eta0):
    def _reflectance(param, eta0):
        return (np.abs((eta0*param[0]-param[1])/(eta0*param[0]+param[1]))**2)[0]
    return tuple(map(_reflectance,param,eta0))

In [12]:
# test
eta0 = admittance(1.0,1.0,0)
eta = admittance(1.5,1.0,0)
param = (np.array([[1],[eta[0]]]),np.array([[1],[eta[1]]]))
ref= reflectance(param,eta0)
#np.isclose(ref,(0.04,0.04))
ref

(0.040000000000000015, 0.040000000000000015)

## マトリクス法
$$
\left(\begin{array}{c}
    B \\
    C
\end{array}\right) = 
\prod_{i=1}^{n}M_{i}
\left(\begin{array}{c}
    1 \\
    \eta_{s}
\end{array}\right)
$$

In [13]:
# code
def calc_matrix(layers,n0,angle,lam):
    mat = (np.eye(2),np.eye(2))
    for layer in layers:
        mat = tuple(map(lambda m1,m2:np.dot(m1,m2),mat,characteristic_matrix(layer[lam],layer['d'],n0[lam],angle,lam)))
    return mat

In [14]:
# test
n0 = {550:1.0}
n_sub = {550:1.5}
layers =[{'d':100, 550:1.0},{'d':100, 550:1.5}]
lam = 550

mat = calc_matrix(layers,n0,0,lam)

eta_sub= admittance(n_sub[lam],n0[lam],0.0)
vec = (np.array([[1],[eta_sub[0]]]),np.array([[1],[eta_sub[1]]]))
param = tuple(map(lambda m,v:np.dot(m,v),mat,vec))

eta0= admittance(n0[lam],n0[lam],0.0)
reflectance(param,eta0)

(0.040000000000000036, 0.040000000000000036)

## 透過
$$
T = \frac{4\eta_{0}\Re(\eta_{s})}{\left|\eta_{0}B+C\right|^{2}}
$$

In [15]:
# code
def transmittance(param,eta0,eta_s):
    def _transmittance(param,eta0,eta_s):
        return (4*eta0*eta_s.real/np.abs(eta0*param[0]+param[1])**2)[0]
    return tuple(map(_transmittance, param, eta0,eta_s))

In [16]:
# test
n0 = {550:1.0}
n_sub = {550:1.5}
layers =[{'d':100, 550:1.0},{'d':100, 550:1.5}]
lam = 550

mat = calc_matrix(layers,n0,0,lam)

eta_sub= admittance(n_sub[lam],n0[lam],0.0)
vec = (np.array([[1],[eta_sub[0]]]),np.array([[1],[eta_sub[1]]]))
param = tuple(map(lambda m,v:np.dot(m,v),mat,vec))

eta0= admittance(n0[lam],n0[lam],0.0)
transmittance(param,eta0,eta_sub)

(0.9600000000000001, 0.9600000000000001)